In [57]:
import pandas as pd

In [58]:

api = pd.read_csv("api_source.csv", parse_dates=['txn_date'])
bank = pd.read_csv("bank_settlement.csv", parse_dates=['txn_date', 'settlement_date'])

print("API columns:", api.columns)
print("BANK columns:", bank.columns)
print(api.head())
print(bank.head())


API columns: Index(['txn_id', 'utr', 'amount', 'txn_date', 'status'], dtype='object')
BANK columns: Index(['txn_id', 'utr', 'amount', 'txn_date', 'status', 'settlement_date'], dtype='object')
   txn_id           utr  amount   txn_date   status
0       1  LEWCTMBSBPCC    1299 2025-11-03  SUCCESS
1       2  P4DHW8UN9B5K     299 2025-10-26  SUCCESS
2       3  EDK4FUWMTCBG     999 2025-11-03  SUCCESS
3       4  XPLUQK3YHUS6     999 2025-10-29  SUCCESS
4       5  ZK9DP2ERBX2U    1299 2025-11-02  SUCCESS
   txn_id           utr  amount   txn_date   status settlement_date
0       1  LEWCTMBSBPCC    1299 2025-11-03  SUCCESS      2025-11-03
1       2  P4DHW8UN9B5K     299 2025-10-26  SUCCESS      2025-10-27
2       3  EDK4FUWMTCBG     999 2025-11-03   FAILED      2025-11-04
3       4  XPLUQK3YHUS6     999 2025-10-29  SUCCESS      2025-10-29
4       5  ZK9DP2ERBX2U    1299 2025-11-02  SUCCESS      2025-11-02


In [59]:
#1)	Ingest	both files standardize dtypes; ensure date parsing

api.columns = api.columns.str.lower().str.strip()
bank.columns = bank.columns.str.lower().str.strip()


api['utr'] = api['utr'].astype(str).str.strip()
bank['utr'] = bank['utr'].astype(str).str.strip()

api['status'] = api['status'].str.lower().str.strip()
bank['status'] = bank['status'].str.lower().str.strip()




In [60]:
#2) 

api_exact  = api.drop_duplicates(subset=['utr', 'amount', 'txn_date']).copy()
bank_exact = bank.drop_duplicates(subset=['utr', 'amount', 'txn_date']).copy()

exact_match = pd.merge(
    api_exact,bank_exact,
    on=['utr', 'amount', 'txn_date'],
    how='inner',
    suffixes=('_api', '_bank')
)




In [61]:
exact_match['category'] = 'exact_match'

print("Exact matches:", len(exact_match))
display(exact_match.head())

exact_match.to_csv('exact_match.csv', index=False)

Exact matches: 72


,txn_id_api,utr,amount,txn_date,status_api,txn_id_bank,status_bank,settlement_date,category
0,1,LEWCTMBSBPCC,1299,2025-11-03,success,1,success,2025-11-03,exact_match
1,2,P4DHW8UN9B5K,299,2025-10-26,success,2,success,2025-10-27,exact_match
2,3,EDK4FUWMTCBG,999,2025-11-03,success,3,failed,2025-11-04,exact_match
3,4,XPLUQK3YHUS6,999,2025-10-29,success,4,success,2025-10-29,exact_match
4,5,ZK9DP2ERBX2U,1299,2025-11-02,success,5,success,2025-11-02,exact_match


In [62]:
#3rd ka 2

amount_mismatch = pd.merge(
    api,
    bank,
    on=['utr', 'txn_date'],
    how='inner',
    suffixes=('_api', '_bank')
)

amount_mismatch = amount_mismatch[amount_mismatch['amount_api'] != amount_mismatch['amount_bank']].copy()
amount_mismatch['category'] = 'amount_mismatch'

print("Amount mismatches:", len(amount_mismatch))
display(amount_mismatch.head())


Amount mismatches: 6


,txn_id_api,utr,amount_api,txn_date,status_api,txn_id_bank,amount_bank,status_bank,settlement_date,category
18,19,EE63E49XMTEA,1299,2025-10-26,success,19,1294,success,2025-10-27,amount_mismatch
20,21,JPE7MQU6P7ST,1499,2025-11-01,success,21,1504,success,2025-11-02,amount_mismatch
28,31,JESHDFBGKK2K,299,2025-11-01,success,31,289,success,2025-11-01,amount_mismatch
31,34,NMBECZJFC45X,999,2025-11-02,success,34,994,success,2025-11-03,amount_mismatch
45,48,D4K7HJSGM863,999,2025-10-29,success,48,1004,success,2025-10-29,amount_mismatch


In [63]:
#3 rd ka 3 status mismatch

status_mismatch = pd.merge(
    api,
    bank,
    on=['utr', 'amount', 'txn_date'],
    how='inner',
    suffixes=('_api', '_bank')
)

status_mismatch = status_mismatch[status_mismatch['status_api'] != status_mismatch['status_bank']].copy()
status_mismatch['category'] = 'status_mismatch'

print(" Stattus mismatches:", len(status_mismatch))
display(status_mismatch.head())

 Stattus mismatches: 4


,txn_id_api,utr,amount,txn_date,status_api,txn_id_bank,status_bank,settlement_date,category
2,3,EDK4FUWMTCBG,999,2025-11-03,success,3,failed,2025-11-04,status_mismatch
29,36,KMANRSGSAJCN,199,2025-11-01,success,36,failed,2025-11-02,status_mismatch
42,50,7ERLKZ9JWKTN,1499,2025-11-02,success,50,failed,2025-11-02,status_mismatch
53,61,X8ENG9EBBN66,999,2025-10-27,success,61,failed,2025-10-27,status_mismatch


In [64]:
#3 rd ka 4th 

missing_in_bank = api.merge(
    bank[['utr']],
    on='utr',
    how='left',
    indicator=True
)

missing_in_bank = missing_in_bank[missing_in_bank['_merge'] == 'left_only'].copy()
missing_in_bank['category'] = 'missing_in_bank'
missing_in_bank = missing_in_bank.drop(columns=['_merge'])

print(" missing in Bank:", len(missing_in_bank))
display(missing_in_bank.head())


missing_in_api = bank.merge(
    api[['utr']],
    on='utr',
    how='left',
    indicator=True
)

missing_in_api = missing_in_api[missing_in_api['_merge'] == 'left_only'].copy()
missing_in_api['category'] = 'missing_in_api'
missing_in_api = missing_in_api.drop(columns=['_merge'])

print(" Missing in API:", len(missing_in_api))
display(missing_in_api.head())

 missing in Bank: 2


,txn_id,utr,amount,txn_date,status,category
23,24,VSSYQTR8Y68J,1499,2025-10-25,success,missing_in_bank
25,26,EYXE68H8NR94,999,2025-10-27,failed,missing_in_bank


 Missing in API: 2


,txn_id,utr,amount,txn_date,status,settlement_date,category
23,24,H6BTNHB2AT8E,1499,2025-10-25,success,2025-10-26,missing_in_api
25,26,GVSW4FKKB63Y,999,2025-10-27,failed,2025-10-28,missing_in_api


In [65]:
exact_match.to_csv('exact_match.csv', index=False)
amount_mismatch.to_csv('amount_mismatch.csv', index=False)
status_mismatch.to_csv('status_mismatch.csv', index=False)
missing_in_bank.to_csv('missing_in_bank.csv', index=False)
missing_in_api.to_csv('missing_in_api.csv', index=False)

#sumamrty
total_api = len(api)
total_bank = len(bank)
exact_cnt = len(exact_match)
amt_mm_cnt = len(amount_mismatch)
status_mm_cnt = len(status_mismatch)
miss_bank_cnt = len(missing_in_bank)
miss_api_cnt = len(missing_in_api)

summary = pd.DataFrame({
    'Category': [
        'exact_match',
        'amount_mismatch',
        'status_mismatch',
        'missing_in_bank',
        'missing_in_api'
    ],
    'Count': [
        exact_cnt,
        amt_mm_cnt,
        status_mm_cnt,
        miss_bank_cnt,
        miss_api_cnt
    ]
})

summary['Share_of_API_%'] = (summary['Count'] / total_api * 100).round(2)

overall_match_rate = round(exact_cnt / total_api * 100, 2) if total_api else 0.0
summary.loc[len(summary)] = ['overall_match_rate', exact_cnt, overall_match_rate]

summary['Total_API'] = total_api
summary['Total_Bank'] = total_bank

summary.to_csv('summary.csv', index=False)

print("categoryy CSVs and summary.csv")
display(summary)

categoryy CSVs and summary.csv


,Category,Count,Share_of_API_%,Total_API,Total_Bank
0,exact_match,72,90.0,80,80
1,amount_mismatch,6,7.5,80,80
2,status_mismatch,4,5.0,80,80
3,missing_in_bank,2,2.5,80,80
4,missing_in_api,2,2.5,80,80
5,overall_match_rate,72,90.0,80,80


In [66]:
#5)

exact_match['txn_date'] = pd.to_datetime(exact_match['txn_date'], errors='coerce')
exact_match['settlement_date'] = pd.to_datetime(exact_match['settlement_date'], errors='coerce')


exact_match['tat_days'] = (exact_match['settlement_date'] - exact_match['txn_date']).dt.days

#statsts
print(" TAT Distribution (in days):")
print(exact_match['tat_days'].describe())

exact_match.to_csv('exact_match_with_tat.csv', index=False)


 TAT Distribution (in days):
count    72.000000
mean      0.486111
std       0.503315
min       0.000000
25%       0.000000
50%       0.000000
75%       1.000000
max       1.000000
Name: tat_days, dtype: float64


In [ ]:
#6)

print(" Final Insights & Observations\n")

#summartyy
total_api = len(api)
total_bank = len(bank)
exact_cnt = len(exact_match)
amt_mis = len(amount_mismatch)
status_mis = len(status_mismatch)
miss_bank = len(missing_in_bank)
miss_api = len(missing_in_api)

match_rate = round((exact_cnt / total_api) * 100, 2) if total_api else 0.0

print(f"Total API transactions     : {total_api}")
print(f"Total Bank transactions    : {total_bank}")
print(f"Exact Matches              : {exact_cnt}")
print(f"Overall Match Rate         : {match_rate}%")
print(f"Amount mismatches          : {amt_mis}")
print(f"Status mismatches          : {status_mis}")
print(f"Missing in Bank            : {miss_bank}")
print(f"Missing in API             : {miss_api}\n")

#  mismatch cuasee
print(" Top 3 causes of mismatches observed:")
print("1️) Amount differences due to rounding or partial settlements between API and Bank")
print("2️) Status mismatches  transactions marked 'success' in API but still 'pending' or 'failed' in Bank")
print("3️3) Missing records  delay or sync issue causing entries to appear in one system but not the other")



 Final Insights & Observations

Total API transactions     : 80
Total Bank transactions    : 80
Exact Matches              : 72
Overall Match Rate         : 90.0%
Amount mismatches          : 6
Status mismatches          : 4
Missing in Bank            : 2
Missing in API             : 2

🔍 Top 3 causes of mismatches observed:
1️) Amount differences due to rounding or partial settlements between API and Bank
2️) Status mismatches  transactions marked 'success' in API but still 'pending' or 'failed' in Bank
3️3) Missing records  delay or sync issue causing entries to appear in one system but not the other
